In [1]:
import tensorflow as tf
%matplotlib notebook
%matplotlib inline
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import cv2
import datetime
import random
from tensorflow.keras.layers import Dense, GRU, TimeDistributed, BatchNormalization, InputLayer

from IPython import display

from tqdm.notebook import tqdm, trange
import seaborn as sns
# sns.set()
tf.keras.backend.floatx()

latent_dim = 256
batch_size = 16
sequence_len = 32 * 4 # n seconds
image_shape = [128, 128, 3]

cvae_max_batch = 64

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [3]:
from convolutional_vae_256 import CVAE
cvae = CVAE(latent_dim)
cvae.load_weights('models/cvae_wide_lat256_epoch620')

In [4]:
def video_generator(video_path):
    while True:
        vidcap = cv2.VideoCapture(video_path)
        success, img = vidcap.read()

        while success:
            yield cv2.resize(img, (128, 128)).astype(np.float32) / 255
            success, img = vidcap.read()

def sequences_generator(video_gen, sequence_len):
    sample = next(video_gen)
    while True:
        sequence = np.zeros((sequence_len, *sample.shape))
        for i in range(sequence_len):
            sequence[i] = next(video_gen)
        yield sequence
            
def batches_generator(batch_size, sequences_gen, buffer_size=64):
    buffer = []
    while True:
        while len(buffer) < buffer_size:
            buffer.append(next(sequences_gen))
        random.shuffle(buffer)
        batch, buffer = buffer[:batch_size], buffer[batch_size:]
        
        # Feed to cvae in small batches (can't fit all sequence in gpu)
        batch = np.array(batch).reshape([-1, *image_shape])
        splits = np.array_split(batch, np.ceil(len(batch) / cvae_max_batch))
        final_batch = []
        for subbatch in splits:
            subbatch = cvae.encode(subbatch)[0]
            final_batch.append(subbatch)
            
        final_batch = np.concatenate(final_batch)
        yield final_batch.reshape(-1, sequence_len, latent_dim)
        

train_gen = batches_generator(batch_size, 
                              sequences_generator(video_generator('data/train.mp4'),
                                                  sequence_len),
                              buffer_size=512)
test_gen = batches_generator(batch_size, 
                             sequences_generator(video_generator('data/test.mp4'),
                                                 sequence_len))

In [5]:
next(train_gen).shape


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(16, 128, 256)

In [6]:
if False:
    %matplotlib notebook
    %matplotlib notebook

    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation

    def roll_demo():
        batch = next(train_gen)
        for entry in batch:
            for frame in entry:
                yield cvae.decode(frame[np.newaxis], apply_sigmoid=True)[0]

    demo_gen = roll_demo()
    im = plt.imshow(next(demo_gen))

    def update(frame):
        plt.title(frame)
        im.set_array(next(demo_gen))
        return [im]

    anim = FuncAnimation(plt.gcf(), update, frames=10000, interval=10, blit=True)

In [7]:
dataset = []


In [8]:
# dataset = dataset.tolist()

In [9]:
for i in trange(387301 // (batch_size * sequence_len)):
    dataset.append(next(train_gen))

In [10]:
dataset = np.array(dataset).reshape([-1, sequence_len, latent_dim])

In [11]:
# dataset = np.concatenate([np.array(dataset[:-790]), np.array(dataset[-790:]).reshape([-1, sequence_len, latent_dim])])

In [12]:
dataset = np.array(dataset).reshape([-1, sequence_len, latent_dim])

file_name = f'data_encoded/lat{latent_dim}_encoded_seqs{len(dataset)}'
np.save(file_name, dataset)

In [13]:
np.load(file_name + '.npy').shape

(3024, 128, 256)

In [14]:
file_name

'data_encoded/lat256_encoded_seqs3024'